In [1]:
import numpy as np
from module import *

In [2]:
n=10**7

In [3]:
unpack_arr=np.random.randint(0,1,size=(n,64),dtype=np.bool_)
arr=np.random.randint(0,2**64-1,size=(n),dtype=np.uint64)
arr_sp=np.random.randint(0,2**16-1,size=(n,4),dtype=np.uint16)

In [4]:
%%timeit
unpack_arr.sum(axis=1)

379 ms ± 5.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
algo_bitcount(arr)

417 ms ± 5.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
algo_bitcount2(arr)

264 ms ± 7.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
algo_bitcount3(arr)

199 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
precalc_bitcount_16bit(arr_sp).sum(axis=1)

158 ms ± 873 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
import cupy as cp

In [4]:
x=cp.random.randint(0,2**32-1,size=(n),dtype=cp.uint32)
y=cp.random.randint(0,2**64-1,size=(n),dtype=cp.uint64)

In [5]:
%%timeit
better_bitcount_32bit(y)

3.67 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
%%timeit
better_bitcount(y)

4.29 ms ± 86.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
pros32 = cp.ElementwiseKernel(
   'uint32 x',
   'uint32 z',
   '''
   z = (x & 0x55555555) + (x >> 1 & 0x55555555);
   z = (z & 0x33333333) + (z >> 2 & 0x33333333);
   z = (z & 0x0F0F0F0F) + (z >> 4 & 0x0F0F0F0F);
   z = (z & 0x00FF00FF) + (z >> 8 & 0x00FF00FF);
   z = (z & 0x0000FFFF) + (z >> 16 & 0x0000FFFF);
   ''',
   'pros32')

In [8]:
%%timeit
pros32(x)

108 µs ± 7.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
pros64 = cp.ElementwiseKernel(
    "uint64 x",
    "uint64 z",
    """
   z = (x & 0x5555555555555555) + (x >> 1 & 0x5555555555555555);
   z = (z & 0x3333333333333333) + (z >> 2 & 0x3333333333333333);
   z = (z & 0x0F0F0F0F0F0F0F0F) + (z >> 4 & 0x0F0F0F0F0F0F0F0F);
   z = (z & 0x00FF00FF00FF00FF) + (z >> 8 & 0x00FF00FF00FF00FF);
   z = (z & 0x0000FFFF0000FFFF) + (z >> 16 & 0x0000FFFF0000FFFF);
   z = (z & 0x00000000FFFFFFFF) + (z >> 32 & 0x00000000FFFFFFFF);
   """,
    "pros64",
)

In [14]:
%%timeit
pros64(y)
cp.cuda.Stream.null.synchronize()

228 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [2]:
%%timeit
(1145141919).bit_count()

19.5 ns ± 0.0866 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [3]:
%%timeit
bin(1145141919).count("1")

151 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
